In [37]:
import mediapipe as mp
import cv2
import numpy as np

In [38]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Make Basic Detection

In [39]:
# Getting video feed
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False 

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True 
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        # Rendering
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2),
                                  )

        cv2.imshow('Raw Webcam Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[x: 0.54858214
y: 0.49424362
z: -1.4083561
visibility: 0.9997496
, x: 0.5752737
y: 0.42760384
z: -1.3093172
visibility: 0.99944884
, x: 0.5991886
y: 0.43163955
z: -1.3101963
visibility: 0.9993389
, x: 0.6208201
y: 0.4358015
z: -1.3099369
visibility: 0.99947256
, x: 0.508493
y: 0.4226898
z: -1.3150852
visibility: 0.99967325
, x: 0.48331147
y: 0.4241829
z: -1.3163784
visibility: 0.9997118
, x: 0.4647506
y: 0.42660522
z: -1.317132
visibility: 0.99976474
, x: 0.632863
y: 0.4701147
z: -0.74388176
visibility: 0.9995907
, x: 0.42626485
y: 0.46827936
z: -0.7601934
visibility: 0.99947256
, x: 0.58924943
y: 0.5746801
z: -1.186063
visibility: 0.9997147
, x: 0.50178957
y: 0.5734693
z: -1.1995878
visibility: 0.9997534
, x: 0.8117198
y: 0.83720696
z: -0.43285412
visibility: 0.9996731
, x: 0.26565436
y: 0.9082579
z: -0.454362
visibility: 0.9994592
, x: 0.9781038
y: 1.0456105
z: -0.6362986
visibility: 0.14313073
, x: 0.14711604
y: 1.3344724
z: -0.5352323
visibility: 0.20512263
, x: 1.1132421
y: 1.5673

# 2. Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [40]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]

x: 0.80439836
y: 0.8535497
z: -0.3251435
visibility: 0.9993859

In [41]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.936358
y: 1.0387012
z: -0.38762212
visibility: 0.15696545

# 3. Calculate Angles

In [42]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [43]:
shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]


In [44]:
shoulder, elbow, wrist

([0.8043983578681946, 0.8535497188568115],
 [0.9363579750061035, 1.0387011766433716],
 [1.0480390787124634, 1.5421617031097412])

In [45]:
calculate_angle(shoulder, elbow, wrist)

157.02930516472196

In [55]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # or any height you prefer
stage = ""
counter = 0
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, 
                  min_tracking_confidence=0.5,
                  model_complexity=2,
                  static_image_mode=False,
                  smooth_landmarks=False,
                          ) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        frame = cv2.flip(frame, 1)
        
        # Extract landmarks
        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for i, landmark in enumerate(landmarks):
                print(f"Landmark {i}: x={landmark.x}, y={landmark.y}, z={landmark.z}")

            # Get coordinates
            angles_to_calculate = {
                "right_elbow_right_shoulder_right_hip": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                ],
                "left_elbow_left_shoulder_left_hip": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                ],
                "right_knee_mid_hip_left_knee": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
                     (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                ],
                "right_hip_right_knee_right_ankle": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
                ],
                "left_hip_left_knee_left_ankle": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
                ],
                "right_wrist_right_elbow_right_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
                ],
                "left_wrist_left_elbow_left_shoulder": [
                    [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
                    [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
                ],
            }

            # Calculate and visualize angles
            angles = []
            for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
                angle = calculate_angle(points[0], points[1], points[2])
                angles.append(angle)
                cv2.putText(image, f'{angle_name}: {int(angle)}', 
                            (50, 100 + i * 30), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

            if angles[3] > 160 and angles[4] > 160:
                stage = "up"
            if angles[3] < 90 and angles[4] < 90 and stage == "up":
                stage = "down"
                counter += 1
        else:
            print("No landmarks detected in this frame.")

            
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        # cv2.imshow('Mediapipe Feed', cv2.flip(image,1))
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Landmark 0: x=0.4702773690223694, y=0.15849798917770386, z=-0.3804865777492523
Landmark 1: x=0.4782679080963135, y=0.14082777500152588, z=-0.36387500166893005
Landmark 2: x=0.48315325379371643, y=0.1415690779685974, z=-0.3641643524169922
Landmark 3: x=0.48749980330467224, y=0.14225435256958008, z=-0.3641456961631775
Landmark 4: x=0.46559539437294006, y=0.13972234725952148, z=-0.36318036913871765
Landmark 5: x=0.46174436807632446, y=0.1402212381362915, z=-0.3634771406650543
Landmark 6: x=0.45849937200546265, y=0.14085108041763306, z=-0.3636256456375122
Landmark 7: x=0.49521422386169434, y=0.15213531255722046, z=-0.24251088500022888
Landmark 8: x=0.45348986983299255, y=0.15199512243270874, z=-0.23716768622398376
Landmark 9: x=0.47938066720962524, y=0.17959144711494446, z=-0.33110693097114563
Landmark 10: x=0.4641347825527191, y=0.17810970544815063, z=-0.33149588108062744
Landmark 11: x=0.524814784526825, y=0.27284467220306396, z=-0.1451554000377655
Landmark 12: x=0.41873517632484436, y=0

In [47]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import torch
# import nbimporter  # Import nbimporter

# from data import NeuralNetwork  
#  # Import your model class

# model = NeuralNetwork()  # Initialize the model
# model.load_state_dict(torch.load(r'D:\PROGRAMMING\BE proj24\data.pth'))  # Load the state dict
# model.eval() 

# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Get video feed
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # or any width you prefer
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720) 
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Make detection
#         results = pose.process(image)

#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract landmarks
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark

#             # Prepare input data for your model
#             input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

#             # Predict next movement
#             with torch.no_grad():
#                 predicted_angles = model(input_tensor)
#                 predicted_angles = predicted_angles.squeeze().tolist()

#             # Display predicted movement
#             for i, angle in enumerate(predicted_angles):
#                 cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', 
#                             (50, 300 + i * 30),  # Adjust y-position to fit all angles
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA) 


#         # Render pose landmarks
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()


In [ ]:
# import sys
# sys.path.append(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.ipynb')
# import nbimporter   # Import nbimporter
# from data import NeuralNetwork

ImportError: cannot import name 'NeuralNetwork' from 'data' (unknown location)

In [ ]:
# import mediapipe as mp
# import cv2
# import numpy as np
# import torch
# import nbimporter   # Import nbimporter
# from data import NeuralNetwork  # Import your model class
# # Initialize the model
# model = NeuralNetwork()
# model.load_state_dict(torch.load(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.pth'))
# model.eval()

# # Set up MediaPipe drawing and pose
# mp_drawing = mp.solutions.drawing_utils
# mp_pose = mp.solutions.pose

# # Open video feed
# cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
# stage = ""
# counter = 0

# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2, smooth_landmarks=False) as pose:
#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         # Recolor image to RGB
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False

#         # Make pose detection
#         results = pose.process(image)

#         # Recolor back to BGR
#         image.flags.writeable = True
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

#         # Extract landmarks and calculate angles
#         if results.pose_landmarks:
#             landmarks = results.pose_landmarks.landmark

#             # Define angle calculations (you can add other angles here)
#             angles_to_calculate = {
#                 "right_elbow_right_shoulder_right_hip": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                 ],
#                 "left_elbow_left_shoulder_left_hip": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                 ],
#                 "right_knee_mid_hip_left_knee": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x) / 2,
#                      (landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y + landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y) / 2],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                 ],
#                 "right_hip_right_knee_right_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y],
#                 ],
#                 "left_hip_left_knee_left_ankle": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y],
#                 ],
#                 "right_wrist_right_elbow_right_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y],
#                 ],
#                 "left_wrist_left_elbow_left_shoulder": [
#                     [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y],
#                     [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, 
#                      landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y],
#                 ],
#             }

#             # Calculate angles and store them in the list for model input
#             angles = []
#             for i, (angle_name, points) in enumerate(angles_to_calculate.items()):
#                 angle = calculate_angle(points[0], points[1], points[2])
#                 angles.append(angle)
#                 cv2.putText(image, f'{angle_name}: {int(angle)}', (50, 100 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)

#             # Prepare input data for the model (use angles as input tensor)
#             input_tensor = torch.tensor(angles, dtype=torch.float32).unsqueeze(0)

#             # Predict next movement
#             with torch.no_grad():
#                 predicted_angles = model(input_tensor).squeeze().tolist()

#             # Display predicted next movement angles
#             for i, angle in enumerate(predicted_angles):
#                 cv2.putText(image, f'Next Angle {i+1}: {int(angle)}', (50, 300 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2, cv2.LINE_AA)

#             # Rep counting logic
#             if angles[3] > 160 and angles[4] > 160:
#                 stage = "up"
#             if angles[3] < 90 and angles[4] < 90 and stage == "up":
#                 stage = "down"
#                 counter += 1

#         # Render rep counter and stage data
#         cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
#         cv2.putText(image, 'REPS', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, str(counter), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
#         cv2.putText(image, 'STAGE', (65, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#         cv2.putText(image, stage, (60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

#         # Render pose landmarks
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
#                                   mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
#                                   mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#         # Display image
#         cv2.imshow('Mediapipe Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()


NameError: name 'model' is not defined

In [ ]:
# import os
# print(os.path.isfile(r'D:\PROGRAMMING\New folder\Rehabilitation-System\data.ipynb'))

True


# Holistic

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# SPINE ANGLE

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Holistic

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_face_mesh = mp.solutions.face_mesh_connections  # Import face_mesh for FACE_CONNECTIONS

cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5,
                          static_image_mode=False,
                          smooth_landmarks=True,
                          model_complexity=2
                            ) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(
            image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION,  # Use FACE_CONNECTIONS from face_mesh module
            mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
            mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(
            image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
        )

        # 3. Left Hand
        mp_drawing.draw_landmarks(
            image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
        )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
        )
                        
        cv2.imshow('Holistic Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# SPINE ANGLE

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calculate_angle(a, b, c):
    a = np.array(a)  # First point  
    b = np.array(b)  # Midpoint
    c = np.array(c)  # End point
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5, model_complexity=2) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates for back straightness check
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            # Calculate angle for back straightness
            spine_angle = calculate_angle(shoulder, hip, knee)
            
            # Visualize spine angle
            cv2.putText(image, f'Spine Angle: {int(spine_angle)}', 
                        (50, 50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Get foot positions for feet check
            left_heel_z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            right_heel_z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z
            
            # Check if feet are lifting off the ground
            if left_heel_z > 0.25 or right_heel_z > 0.25:  # Adjust threshold as needed
                cv2.putText(image, "Feet not planted!", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, "Feet planted", 
                            (50, 100), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                       
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
        
        cv2.imshow('Squat Form Detection', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
